# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [22]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [23]:
#!pip install pandas

In [24]:
import pandas as pd
df = pd.read_csv('data/Data.csv')

In [25]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [1]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [11]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.5)

#balanced

In [12]:

prompt = ChatPromptTemplate.from_template(
    "Write a compelling product description for a {product_name}. Highlight its features, benefits, and ideal use cases."
)


In [13]:

chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
product = "electric car"
chain.run(product)

"Introducing the future of transportation with our cutting-edge electric cars. Say goodbye to gas-guzzling vehicles and hello to eco-friendly, efficient driving. \n\nOur electric cars are equipped with state-of-the-art technology that allows for seamless integration with your daily life. With fast charging capabilities, you can hit the road with confidence knowing you won't be left stranded. Plus, the quiet and smooth ride will make you forget you're even driving an electric vehicle.\n\nBut the benefits don't stop there. By choosing an electric car, you're not only reducing your carbon footprint, but you're also saving money on gas and maintenance costs. With fewer moving parts, electric cars require less upkeep, meaning more money in your pocket in the long run.\n\nIdeal for city commuters, families, and environmentally conscious individuals, our electric cars are the perfect solution for those looking to make a positive impact on the planet without sacrificing style or performance. S

## SimpleSequentialChain

In [15]:
from langchain.chains import SimpleSequentialChain

In [18]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
     "Describe the following product in detail: {product}"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [19]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a marketing tagline for the following product description: {input}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [20]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [21]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Electric cars, also known as electric vehicles (EVs), are automobiles powered by electricity instead of gas. They use one or more electric motors for propulsion, and are typically powered by rechargeable batteries.

These cars produce zero tailpipe emissions, making them much more environmentally friendly than traditional gasoline-powered vehicles. This can help reduce air pollution and greenhouse gas emissions, making electric cars a popular choice for those looking to reduce their carbon footprint.

Electric cars are also known for their quiet operation and smooth acceleration. They often have regenerative braking systems, which can help to extend the range of the vehicle by recapturing energy that would otherwise be lost during braking.

With advances in battery technology, the range of electric cars has been steadily increasing, with some models now able to travel hundreds of miles on a single charge. Charging infrastructure is also i

'"Drive cleaner, drive smarter with electric cars - the eco-friendly choice for a sustainable future."'

**Repeat the above twice for different products**

## SequentialChain

In [26]:
from langchain.chains import SequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.9)


desc_prompt = ChatPromptTemplate.from_template(
    "Write a detailed product description for: {product}"
)
desc_chain = LLMChain(llm=llm, prompt=desc_prompt, output_key="description")


In [ ]:
# Step 2: Generate slogan
slogan_prompt = ChatPromptTemplate.from_template(
    "Create a short slogan for the following: {description}"
)
slogan_chain = LLMChain(llm=llm, prompt=slogan_prompt, output_key="slogan")


In [ ]:
# Step 3: Marketing email from description and slogan
email_prompt = ChatPromptTemplate.from_template(
    "Write a marketing email for a product with the following description: {description} and slogan: {slogan}"
)
email_chain = LLMChain(llm=llm, prompt=email_prompt, output_key="email")



In [ ]:
# Combine chains with named variables
seq_chain = SequentialChain(
    chains=[desc_chain, slogan_chain, email_chain],
    input_variables=["product"],
    output_variables=["description", "slogan", "email"],
    verbose=True,
)

In [32]:
#review = df.Review[5]
#overall_chain(review)

product = "dishwasher"
seq_chain (product)



> Entering new SequentialChain chain...

> Finished chain.


{'product': 'dishwasher',
 'description': "A dishwasher is an essential appliance in any modern kitchen, designed to efficiently clean and sanitize your dirty dishes, glassware, and utensils. This time-saving device eliminates the need for hand-washing, making it a convenient and hygienic solution for busy households.\n\nThe typical dishwasher consists of a series of racks and compartments for organizing your dishes, along with a spray arm and detergent dispenser for thorough cleaning. Some models also feature adjustable racks, specialized wash cycles, and energy-saving options to suit your specific needs.\n\nWith a dishwasher, you can say goodbye to the hassle of scrubbing and rinsing dirty dishes, as this appliance uses high-pressure jets of hot water and detergent to remove food debris and grease. Many dishwashers also have a heated drying cycle to ensure your dishes are ready to use right out of the machine.\n\nIn addition to saving time and effort, using a dishwasher can also help

**Repeat the above twice for different products or reviews**

## Router Chain

In [48]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""

biology_template = """You are a very smart biology professor. \
You are great at answering questions about biology in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""



math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [49]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
      {
        "name": "biology", 
        "description": "Good for answering questions about biology", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [37]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [38]:
llm = ChatOpenAI(temperature=0)

In [50]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [51]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [52]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [53]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [54]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [44]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is characterized by a continuous spectrum of wavelengths and intensities, with the peak intensity shifting to shorter wavelengths as the temperature of the black body increases. Black body radiation plays a key role in understanding concepts such as thermal radiation and the quantization of energy in quantum mechanics."

In [45]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [55]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA carries the genetic information that determines the characteristics of an organism. This genetic information is essential for the cell to function properly and carry out its specific roles within the body. DNA also allows for the replication and transmission of genetic information from one generation to the next. In summary, DNA is crucial for the proper functioning and survival of cells in our body.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**